In [ ]:
import sklearn
sklearn.__version__

In [ ]:
import pandas as pd   
from matplotlib import pyplot
from numpy import mean
import time


from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.preprocessing import MinMaxScaler



In [ ]:
df1 = pd.read_csv("train.csv")
df1.head()
df1.drop(['RecordID', 'hospital_id'], axis=1, inplace=True)


In [ ]:
print(df1.shape)
print(df1.dtypes)

In [ ]:
# df_onehot = pd.get_dummies(df1)
# print(df_onehot.dtypes)
# print(df_onehot.shape)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer

def impute_dataframe(df):
    # Identify numerical and categorical columns
    numerical_columns = df.select_dtypes(include=[np.number]).columns
    categorical_columns = df.select_dtypes(exclude=[np.number]).columns

    # Perform mean imputation on numerical columns
    numerical_imputer = SimpleImputer(strategy='mean')
    df[numerical_columns] = numerical_imputer.fit_transform(df[numerical_columns])

    # Create a DataFrame containing only the categorical columns
    categorical_df = df[categorical_columns]

    # Perform one-hot encoding on categorical columns
    categorical_df = pd.get_dummies(categorical_df)

    # Convert all column dtypes to binary (int64)
    categorical_df = categorical_df.astype('int64')

    # Perform K-nearest neighbor imputation on categorical columns
    knn_imputer = KNNImputer(n_neighbors=5)
    categorical_df = knn_imputer.fit_transform(categorical_df)
    categorical_df = pd.DataFrame(categorical_df)

    # Convert the result back to a DataFrame with original column names
    categorical_df = pd.DataFrame(categorical_df, columns=categorical_df.columns)

    # Combine the imputed numerical and categorical DataFrames
    imputed_df = pd.concat([df[numerical_columns], categorical_df], axis=1)

    return imputed_df
 
# imputed_df = impute_dataframe(df1)
# imputed_df.head()
# imputed_df.shape




In [ ]:

def missing_value_counts(dataframe):
    # Calculate the count of missing values in each column
    missing_counts = dataframe.isnull().sum()

    # Create a DataFrame to display the results
    missing_counts_df = pd.DataFrame({'Column': missing_counts.index, 'Missing_Values_Count': missing_counts.values})

    return missing_counts_df

missing_value_counts(imputed_df)

In [ ]:
X = imputed_df.loc[:, imputed_df.columns != "hospital_death"]
y = imputed_df[["hospital_death"]]
# Convert integer column names to strings
X.columns = testX.columns.astype(str)
X.dtypes


In [ ]:
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.1, random_state=2)

 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Create a K-nearest neighbors model with 5 neighbors
knn_model = KNeighborsClassifier(n_neighbors=300)

# Fit the model on the training data
knn_model.fit(trainX, trainy)

md_probs = knn_model.predict_proba(testX)

print(md_probs)

md_probs = md_probs[:,1]
print(md_probs)


In [ ]:
md_auc = roc_auc_score(testy, md_probs)
md_auc

In [ ]:
# def fit_model(model, model_name):
#     model.fit(trainX,trainy)
#     md_probs = model.predict_proba(testX)
#     md_probs = md_probs[:,1]
#     md_auc = roc_auc_score(testy, md_probs)
#     print(model_name, " : ", md_auc)
#     md_fpr, md_tpr, _ = roc_curve(testy, md_probs)
#     pyplot.plot(md_fpr, md_tpr, marker='.', label=model_name)
#     #return (md_fpr, md_tpr)
    

In [ ]:
# def fitting_models_CV2(X, y):
#     cv = RepeatedKFold(n_splits=10, n_repeats=2)
    
#     # dt_5_2 = DecisionTreeClassifier(max_depth=5, )  
#     # dt_7_2 = DecisionTreeClassifier(max_depth=7)  
#     # dt_5_5 = DecisionTreeClassifier(max_depth=5, min_samples_split=5)  
#     # dt_7_5 = DecisionTreeClassifier(max_depth=7, min_samples_split=5)  
#     dt_7_3 = DecisionTreeClassifier(max_depth=7, min_samples_split=3)  
#     clfs = [
#             # ('Depth 5 Split 2', dt_5_2),        
#             # ('Depth 7 Split 2', dt_7_2),        
#             # ('Depth 5 Split 5', dt_5_5),
#             # ('Depth 7 Split 5', dt_7_5),
#             ('Depth 10 Split 2', dt_7_3)
            
#            ]

#     models = []
#     for name, clf in clfs:
#         clf.fit(X, y)  # Fit the model to your data
#         models.append((name, clf))  # Store the trained model
        
#         start = time.perf_counter()
#         accuracy = cross_val_score(clf, X, y, scoring="accuracy", cv=cv, n_jobs=-1)
#         scores = cross_val_score(clf, X, y, scoring="roc_auc", cv=cv, n_jobs=-1) 
#         end = time.perf_counter()        
#         score = format(mean(scores), '.4f')
#         duration = format((end-start), '.4f')
#         accuracy = format(mean(accuracy), '.4f')
        
#         print("{} : score:{} - time taken:{} - accuracy:{}".format(name, score, duration, accuracy))
    
#     return models


In [ ]:
# models = fitting_models_CV2(X,y)


In [ ]:
# models
# for name, model in models:
#     print(type(model))
#     model.predict_proba(testX)
    
    


In [ ]:
# import numpy as np

# def generate_predictions(models, test_file):
#     df_test = pd.read_csv(test_file)
#     df_test_onehot = pd.get_dummies(df_test)
#     X_test = df_test_onehot.loc[:, df_test_onehot.columns != "hospital_death"]
#     record_ids = df_test["RecordID"]
#     predictions = []
#     for name, model in models:
        
#         probs = model.predict_proba(X_test)
#         probs = probs[:,1]
#         predictions.append(probs)
#     predictions = np.array(predictions)
#     predictions = np.mean(predictions, axis=0)
#     result = pd.DataFrame({'RecordID': record_ids, 'hospital_death': predictions})
#     return result


# results = generate_predictions(models, "test.csv")
# results.to_csv("results.csv", index=False, header=["RecordID", "hospital_death"])


In [ ]:
def generate_predictions_for_model(model, test_file, output_file):
    # Load the test data
    df_test = pd.read_csv(test_file)
    df_test_imputed = impute_dataframe(df_test)
    
    X_test = df_test_imputed
    X_test.drop(['RecordID', 'hospital_id'], axis=1, inplace=True)
    X_test.columns = X_test.columns.astype(str)

    record_ids = df_test["RecordID"]
    
    # Generate predictions using the model
    probs = model.predict_proba(X_test)
    probs = probs[:, 1]
    
    # Create a DataFrame for the results
    result = pd.DataFrame({'RecordID': record_ids, 'hospital_death': probs})
    
    # Save the results to a CSV file
    result.to_csv(output_file, index=False, header=["RecordID", "hospital_death"])
    
# generate_predictions_for_model(knn_model, "test.csv", "results.csv")
